In [1]:
from tweepy import Stream, OAuthHandler
from tweepy.streaming import StreamListener
import os
import sys
import time, json, sqlite3, re
from geopy.geocoders import Nominatim
from time import mktime
import gmplot
from datetime import timedelta
from datetime import datetime


In [ ]:
#ConnectingDB
#conn = sqlite3.connect('twitterdemiologyDB.db')
#c = conn.cursor()

#CreatingTable
#c.execute('CREATE TABLE IF NOT EXISTS Main(id TEXT, date DATETIME, lat TEXT, lon TEXT)')

#AccessCodes
consumerKey = "TTbRIgdtpJUcQUSVZe19IurDc"
consumerSecret = "hf54y1TpkhpxkhLQBMmJvbTMfFMMUfGbgYZw5xbswIad3ZWzer"
accessToken = "372330040-UqVzc2sUKDFz8ZDUowAufuyAv5uaxmbvGnXTyhj3"
accessSecret = "6vArqK40I23JURV8Uevek7PNvBqCfY0nhlmVJ2hsQ1zEm"

#Scraping/ParsingTweets
class Listener(StreamListener):

    def on_data(self, raw_data):
        try:
            jsonData = json.loads(raw_data)
            #Converting date to datetime format:
            date = jsonData['created_at']
            date2 = str(date).split(' ')
            date3 = date2[1]+' '+date2[2]+' '+date2[3]+' '+date2[5]
            datetime_object = time.strptime(date3, '%b %d %H:%M:%S %Y')
            dt = datetime.fromtimestamp(mktime(datetime_object))
            #Parsing out ID, the tweet itself, and location:
            tweetID = jsonData['id_str']
            pretweet = jsonData['text']
            userInfo = jsonData['user']
            location = userInfo['location']
            if jsonData['lang'] == 'en' and location != 'Midwest' and location!= 'Whole World' and location != 'Earth':
                print(dt, pretweet, location)
                geolocator = Nominatim()
                geolocation = geolocator.geocode(location)
                print("Lat {} and long {}".format(geolocation.latitued, geolocation.longitude))
                try:
                    #The 2-5 len range helps to remove inaccurate/unspecific locations
                    if "United States of America" in geolocation.address[::] and 5>= len(geolocation.address.split(",")) > 2 :
                        lat = geolocation.latitude
                        lon = geolocation.longitude
                        print(geolocation.address, '\n', lat, lon)
                        if not jsonData['retweeted'] and 'RT @' not in pretweet:
                            tweet = re.sub(r"(http\S+|#\b)", "", pretweet)
                            tweet = " ".join(filter(lambda x: x[0] != '@', tweet.split()))
                            tweet = str(tweet.encode('unicode-escape')).replace('\\', ' ')
                            print(tweet)
                except Exception as e:
                    print("Invalid locaion:", str(e))
        except BaseException as e:
            print("Failed ondata,",str(e))
            time.sleep(0.1)
    def on_error(self, status_code):
        print(status_code)

#Access
auth = OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessSecret)

#InitiateStreaming
twitterStream = Stream(auth, Listener())
twitterStream.filter(track=['flu'])

2020-03-29 16:02:14 RT @oconnorwrites: Lucy Jarratt has a curious to claim to fame. She survived the flu pandemic - of 1918. Over a century later, she is ridin… Toronto, formerly of Augher


/home/loki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Failed ondata, 'NoneType' object has no attribute 'latitude'
2020-03-29 16:02:15 RT @JeffreyGoldberg: I can't get this bit of analysis, from Donald Trump, off my mind:

"You can call it a germ. You can call it a flu. You… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:16 RT @RealMattCouch: I refuse to listen to any fear mongers who wont list the Flu statistics along with the CoronaVirus statistics... 

The C… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:16 @TVandFilmStars Take care of yourself! Have you been tested for flu &amp; Covid-19? Bothell, WA
GEOLOCATYION 47.7598697
Bothell, King County, Washington, United States of America 
 47.7598697 -122.2068288
b'Take care of yourself! Have you been tested for flu &amp; Covid-19?'
2020-03-29 16:02:16 RT @LincolnsBible: The hospital beds are donald’s undoing.
It will be the journalists &amp; nurses/health care workers capturing images of sick… Steilacoom, WA
GEOLOCATYION 47.1703692
Steilacoom, Pierce Co

GEOLOCATYION 12.9791198
Location not in USA
2020-03-29 16:02:26 RT @charliekirk11: This is your daily reminder that the Swine Flu infected 60 million Americans

It resulted in 12,469 American deaths

Doe… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:27 RT @TruthBe53952128: 2009 Pandemic Drill had 60 Million Cases😳and 12,469 deaths... 0.02% in other words it was the flu https://t.co/pEtdPOS… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:27 @epamericasfb You needa be nearly dying for them to give u an actual test, u gotta have flu symptoms, fever &amp; torn… https://t.co/LNYCle79aq Texas, USA
GEOLOCATYION 31.8160381
Location not in USA
2020-03-29 16:02:27 RT @AetiusRF: Found a place that sells desert night and tiger stripe in a crye combat cut but based on its location I have to decide if I w… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:27 RT @DavidWohl: @realDonaldTrump Americans - even Democrats - appreciate Trump's tireless w

GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:33 @disneydude365 @Mikel_Jollett @H_Combs In reality, it could be much worse. Italy has an overall rate of 10% or 100… https://t.co/B8QDtdWzUw None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:34 @FrustratedWeat1 @TSlocumJr @chrisjollyhale COVID 19 is a completely different family of viruses than influenza. So… https://t.co/i2Dfwc3C69 None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:34 @MalcolmNance @RealCandaceO The hospitals are overloaded right now. Its only going to get worse in the coming weeks… https://t.co/6WimYGIcCM None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:35 RT @JustinWolfers: A careful study of the Spanish flu reveals that “the earlier, more forcefully and longer cities responded, the better th… 20 ♥ they ♥ stan bakuiida💥💨
Failed ondata, 'NoneType' object has no attribute 'latitude'
2020-03-29 16:02:35 The name Wohl is generally considered to be synonymous with l

GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:43 RT @DesperateAnnie: A letter from F. Scott Fitzgerald, quarantined in the south of France during the Spanish Flu outbreak (1920).
#Covid_19… Tokyo, Japan
GEOLOCATYION 35.6828387
Location not in USA
2020-03-29 16:02:43 RT @RealCandaceO: Swine Flu epidemic 2009:

61 million Americans infected. 
18,000 Americans died, all ages affected similarly. 
Symptoms:… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:43 @LovesTrump2020 @Manageorge95 @kellyscott929 @butterfly5366 @GovLarryHogan Excuse me but it is NOT just like the se… https://t.co/bxI3hu6Vgh Maryland, USA
GEOLOCATYION 39.5162234
Location not in USA
2020-03-29 16:02:43 RT @RealCandaceO: The Corona doomsday cult refuses all questions about the flu because “IT’S NOT THE FLU, BRO!”

We are simply asking you t… Madrid - Spain
GEOLOCATYION 40.4167047
Location not in USA
2020-03-29 16:02:43 His white blood cells when a virus comes around https://t.co/BvvrY7mGxC 

GEOLOCATYION 35.6828387
Location not in USA
2020-03-29 16:02:52 RT @JoeySalads: 54 Million illnesses
730,000 flu hospitalizations
62,000 flu deaths

This is not the Coronavirus, this is the seasonal flu,… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:02:52 @Woodyman502 @chuckwoolery Weird how this doesn't include the task force he created in January, or the travel ban.… https://t.co/cIMEnbu1AG The West Is The Best
GEOLOCATYION 51.510017
Location not in USA
2020-03-29 16:02:52 RT @AnnetheNurse1: @RealCandaceO I love you, Candace. I really do. But the flu patients are spread out over many months and not concentrate… Houston,  TX 
GEOLOCATYION 29.7589382
Houston, Harris County, Texas, 77002-5902, United States of America 
 29.7589382 -95.3676974
Is a Retweet
2020-03-29 16:02:52 RT @PeterKolchinsky: In the fight, some copies might turn out to have a mix of pages from their reports. A mixed report might have Page 1 f… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-2

GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:03:01 RT @iamjumpingin: All these daily updates, death rate, number who have contracted the virus etc. It can keep you in a state of depression.… None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:03:01 RT @NWPBroadcasting: LONG READ: History feels like it’s repeating itself with the coronavirus outbreak. The 1918 flu pandemic offers lesson… Spokane, Washington
GEOLOCATYION 47.6579425
Spokane, Spokane County, Washington, United States of America 
 47.6579425 -117.4212265
Is a Retweet
2020-03-29 16:03:01 @laporteswand @its_tha_mob It’s worrying that those who are dying are mostly those who have stayed home &amp; not recei… https://t.co/I3VyXntuzm None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:03:02 @ScanlanWithAnA Again they didn't want to make the same mistakes as with Bird Flu - ie too much panic and over the… https://t.co/gnyQVHQSwK Southwest UK
GEOLOCATYION 51.4741779
Location not in USA
2020-03-29 16

GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:03:13 RT @JeffreyGoldberg: I can't get this bit of analysis, from Donald Trump, off my mind:

"You can call it a germ. You can call it a flu. You… United States
GEOLOCATYION 39.7837304
Location not in USA
2020-03-29 16:03:13 RT @spectatorindex: ITALY: 101-year old man who was born during the Spanish Flu pandemic has recovered from a coronavirus infection None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:03:13 @thepineappleend @__NiGhThAwK_727 @CamrynSimmons6 @aroundthegIobe Don’t trust the vaccines or the tests as they tes… https://t.co/FMJVCwVPeI None
GEOLOCATYION 44.933143
Location not in USA
2020-03-29 16:03:13 RT @OteeMaroty_: Definitely a Person of Interest. Could be lifestyle or genetics. Hunkuyi, 🇳🇬 Ibiza
Failed ondata, 'NoneType' object has no attribute 'latitude'
2020-03-29 16:03:14 @AstroKatie @randileeharper Interrupting the animal -&gt; human jump is my guess. They tend to happen because humans c… https://

In [ ]:
#QueryingData
scanner = str('2017-06-22 10:59:25')
for i in range(10): #Number of days that the data spans I just put 10 becuase I don't want a
                    #bunch of html files in my dir (I already made them)
    tail = str(datetime.strptime(scanner, '%Y-%m-%d %H:%M:%S') - timedelta(hours=168))
    c.execute('SELECT lat, lon FROM Main WHERE date > "{x}" AND date < "{y}"'.format(x=tail,y = scanner))
    latArray = []
    lonArray = []
    for row in c.fetchall():
        if row[0] and row[1]:
            latArray.append(float(row[0]))
            lonArray.append(float(row[1]))
    #MappingData
    file = 'flu' + str(scanner) + '.html'
    gmap = gmplot.GoogleMapPlotter.from_geocode("United States")
    gmap.heatmap(latArray, lonArray)
    gmap.draw(file)
    scanner = str(datetime.strptime(scanner, '%Y-%m-%d %H:%M:%S') + timedelta(hours=24))